# Import Tools

In [1]:
import os
import json
import cv2
import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
import torch
import tqdm
import shutil
HOME = os.getcwd()
print(HOME)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

C:\Users\USER\Downloads\DeepFashion2


In [2]:
!pip install -q ultralytics supervision --upgrade supervision
from ultralytics import YOLO
import supervision as sv

In [3]:
DATA_PATH = f"{HOME}/yolov9/colorful-dataset"
IMAGE_PATH = f"{HOME}/yolov9/colorful-dataset/train/images"
WEIGHT_PATH = f"{HOME}/yolov9/weights/yolov9c.pt"
# SINGLE_IMAGE = IMAGE_PATH + "/000001_png.rf.3efc63de2fe2bab8882abc87437bf048.jpg"
SINGLE_IMAGE = "yolov9/mytest/5.jpeg"

images_path = DATA_PATH+'/JPEGImages/'
annotations_path  = DATA_PATH+'/Annotations_txt/'

BEST_WEIGHT = f'{HOME}/runs/detect/train28/weights/best.pt'

# Get the model

In [4]:
model = YOLO(WEIGHT_PATH)
model.info()

YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs


(618, 25590912, 0, 104.02268160000003)

# Get the data

In [5]:
train = []
with open(HOME+"/yolov9/colorful-dataset/ImageSets/Main/trainval.txt", "r") as f:
    for line in f.readlines():
        if line[-1]=='\n':
            line = line[:-1]
        train.append(line)

test = []
with open(HOME+"/yolov9/colorful-dataset/ImageSets/Main/test.txt", "r") as f:
    for line in f.readlines():
        if line[-1]=='\n':
            line = line[:-1]
        test.append(line)

In [6]:
len(train), len(test)

(2145, 537)

In [7]:
# os.mkdir(DATA_PATH+'/train')
# os.mkdir(DATA_PATH+'/train/images')
# os.mkdir(DATA_PATH+'/train/labels')

# os.mkdir(DATA_PATH+'/test')
# os.mkdir(DATA_PATH+'/test/images')
# os.mkdir(DATA_PATH+'/test/labels')

In [8]:
# train_path = DATA_PATH+'/train/'
# test_path = DATA_PATH+'/test/'

# print('Copying Train Data..!!')
# for i in tqdm.tqdm(train):
#     a = shutil.copyfile(images_path+i+'.jpg', train_path+'images/'+i+'.jpg')
#     a = shutil.copyfile(annotations_path+i+'.txt', train_path+'labels/'+i+'.txt')

# print('Copying Test Data..!!')
# for i in tqdm.tqdm(test):
#     a = shutil.copyfile(images_path+i+'.jpg', test_path+'images/'+i+'.jpg')
#     a = shutil.copyfile(annotations_path+i+'.txt', test_path+'labels/'+i+'.txt')

# Train Custom Model

In [9]:
text = """
val: ../colorful-dataset/test/images
train: ../colorful-dataset/train/images

# number of classes
nc: 10

# class names
names: ['sunglass','hat','jacket','shirt','pants','shorts','skirt','dress','bag','shoe']

"""
with open(DATA_PATH+"/data.yaml", 'w') as file:
    file.write(text)

In [10]:
results = model.train(data=DATA_PATH+"/data.yaml", epochs=1000, imgsz=480, patience=200)

Ultralytics YOLOv8.1.40 🚀 Python-3.8.18 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8187MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\USER\Downloads\DeepFashion2/yolov9/weights/yolov9c.pt, data=C:\Users\USER\Downloads\DeepFashion2/yolov9/colorful-dataset/data.yaml, epochs=1000, time=None, patience=200, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train33, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

wandb: Currently logged in as: krittinjj2547 (jwizzed). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning C:\Users\USER\Downloads\DeepFashion2\yolov9\colorful-dataset\train\labels.cache... 2145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2145/2145 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        537       2035      0.774      0.755      0.774      0.532
Stopping training early as no improvement observed in last 200 epochs. Best results observed at epoch 751, best model saved as best.pt.
To update EarlyStopping(patience=200) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



951 epochs completed in 13.072 hours.
Optimizer stripped from runs\detect\train33\weights\last.pt, 51.7MB
Optimizer stripped from runs\detect\train33\weights\best.pt, 51.7MB

Validating runs\detect\train33\weights\best.pt...
Ultralytics YOLOv8.1.40 🚀 Python-3.8.18 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 8187MiB)
YOLOv9c summary (fused): 384 layers, 25326958 parameters, 0 gradients, 102.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:04<00:00,  3.57it/s]


                   all        537       2035      0.826      0.718      0.781      0.541
              sunglass        537         82      0.749       0.28      0.373      0.121
                   hat        537         77       0.81      0.664      0.782      0.451
                jacket        537        181      0.867      0.762      0.863      0.704
                 shirt        537        366      0.842      0.787      0.825      0.612
                 pants        537        114      0.938      0.912      0.955      0.781
                shorts        537        107      0.857       0.84      0.813      0.529
                 skirt        537        186      0.801      0.747      0.813      0.652
                 dress        537        128      0.724       0.78      0.792      0.654
                   bag        537        274       0.83      0.658      0.767       0.44
                  shoe        537        520       0.84      0.746      0.823      0.469
Speed: 0.2ms preproce

lr/pg0,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁█▇▆▇▆▆▆▇▆▇▇▇▇▇▇██▇▇▇▇▇▇████████████████
metrics/mAP50-95(B),▁▇▆▆▆▆▆▆▆▆▇▇▇▇▇▇██████████████████████▇▇
metrics/precision(B),▁▅▅▅▆▄▅▄▆▇▅▇▆▆▆▆▇▇▅▅▅▇▇▆▆▆▆▆▆▆▇█▅█▅▅▅▆▆▆
metrics/recall(B),▁▇▇▆▇▇▆▆▆▄▆▅▆▆▅▆▅▆▇▇▇▅▅▆▆▆▆▆▆▆▆▅█▅███▇█▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
